# SQL Agent with GPT5

In [4]:
from dotenv import load_dotenv
load_dotenv(".env")

True

In [7]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("openai:gpt-5")

In [9]:
import sqlite3

# --- Database setup ---
DB_NAME = "shop.db"

# Connect (creates file if it doesn't exist)
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

# --- Create tables ---
cursor.execute("""
CREATE TABLE IF NOT EXISTS products (
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    description TEXT,
    price REAL NOT NULL,
    stock INTEGER NOT NULL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS orders (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_name TEXT NOT NULL,
    product_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    order_date TEXT DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (product_id) REFERENCES products(product_id)
)
""")

# --- Insert sample data ---
products = [
    ("Laptop", "14-inch display, 8GB RAM, 512GB SSD", 65000.00, 10),
    ("Phone", "6.5-inch AMOLED, 128GB storage", 25000.00, 25),
    ("Headphones", "Wireless Bluetooth headphones", 2000.00, 50),
]
cursor.executemany(
    "INSERT INTO products (name, description, price, stock) VALUES (?, ?, ?, ?)",
    products
)

orders = [
    ("John Doe", 1, 1),  # Ordered 1 Laptop
    ("Jane Smith", 2, 2),  # Ordered 2 Phones
]
cursor.executemany(
    "INSERT INTO orders (customer_name, product_id, quantity) VALUES (?, ?, ?)",
    orders
)

# --- Commit changes and close ---
conn.commit()
conn.close()

print(f"Database '{DB_NAME}' created with sample products and orders.")


Database 'shop.db' created with sample products and orders.


In [10]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///shop.db")

print(f"Dialect: {db.dialect}")
print(f"Available tables: {db.get_usable_table_names()}")
print(f'Sample products: {db.run("SELECT * FROM products LIMIT 5;")}')
print(f'Sample orders: {db.run("SELECT * FROM orders LIMIT 5;")}')

Dialect: sqlite
Available tables: ['orders', 'products']
Sample products: [(1, 'Laptop', '14-inch display, 8GB RAM, 512GB SSD', 65000.0, 10), (2, 'Phone', '6.5-inch AMOLED, 128GB storage', 25000.0, 25), (3, 'Headphones', 'Wireless Bluetooth headphones', 2000.0, 50)]
Sample orders: [(1, 'John Doe', 1, 1, '2025-08-08 10:23:04'), (2, 'Jane Smith', 2, 2, '2025-08-08 10:23:04')]


In [33]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

for i in tools:
    print(i.description)
def list_db_tables():
    """Input is an empty string, output is a comma-separated list of tables in the database."""
    return "table_names: orders, products"

def get_db_schema():
    """Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3"""
    schema = """ CREATE TABLE orders (
    	order_id INTEGER, 
    	customer_name TEXT NOT NULL, 
    	product_id INTEGER NOT NULL, 
    	quantity INTEGER NOT NULL, 
    	order_date TEXT DEFAULT CURRENT_TIMESTAMP, 
    	PRIMARY KEY (order_id), 
    	FOREIGN KEY(product_id) REFERENCES products (product_id)
    )
    
    /*
    3 rows from orders table:
    order_id	customer_name	product_id	quantity	order_date
    1	John Doe	1	1	2025-08-08 10:23:04
    2	Jane Smith	2	2	2025-08-08 10:23:04
    */
    
    
    CREATE TABLE products (
    	product_id INTEGER, 
    	name TEXT NOT NULL, 
    	description TEXT, 
    	price REAL NOT NULL, 
    	stock INTEGER NOT NULL, 
    	PRIMARY KEY (product_id)
    )
    
    /*
    3 rows from products table:
    product_id	name	description	price	stock
    1	Laptop	14-inch display, 8GB RAM, 512GB SSD	65000.0	10
    2	Phone	6.5-inch AMOLED, 128GB storage	25000.0	25
    3	Headphones	Wireless Bluetooth headphones	2000.0	50
    */"""
    return schema

tools = [ i for i in tools if not i.name in ["sql_db_schema", "sql_db_list_tables"]]

tools.append(list_db_tables)
tools.append( get_db_schema)



Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.
Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3
Input is an empty string, output is a comma-separated list of tables in the database.
Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!


In [34]:
from langgraph.prebuilt import create_react_agent

system_prompt = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
""".format(
    dialect=db.dialect,
    top_k=5,
)

agent = create_react_agent(
    llm,
    tools,
    prompt=system_prompt,
)

In [37]:
%%time

# CPU times: total: 203 ms
# Wall time: 34.5 s

# CPU times: total: 156 ms
# Wall time: 25.7 s

question = "How much total revenue did we made from sales?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

How much total revenue did we made from sales?
================================== Ai Message ==================================
Tool Calls:
  list_db_tables (call_fPKRX40XerGoPKXomLMVlCvR)
 Call ID: call_fPKRX40XerGoPKXomLMVlCvR
  Args:
================================= Tool Message =================================
Name: list_db_tables

table_names: orders, products
================================== Ai Message ==================================
Tool Calls:
  get_db_schema (call_ov8tHebsOUNKgW4vPg7B6590)
 Call ID: call_ov8tHebsOUNKgW4vPg7B6590
  Args:
    tables: orders, products
================================= Tool Message =================================
Name: get_db_schema

 CREATE TABLE orders (
    	order_id INTEGER, 
    	customer_name TEXT NOT NULL, 
    	product_id INTEGER NOT NULL, 
    	quantity INTEGER NOT NULL, 
    	order_date TEXT DEFAULT CURRENT_TIMESTAMP, 
    	PRIMARY KEY (order_id), 